Understanding what data products are released about a species is important in understanding what we know about the species and what gaps may exist in current knowledge.  This notebook leverages USGS SB to return official data release items that mention species of interest.

In [22]:
import requests
import json
import bispy
from IPython.display import display
from joblib import Parallel, delayed
import random
from bispy import bis 

sb = bis.Sciencebase()

In [23]:
# Open source WLCI list created from build-specie-list.ipynb
with open("cache/WLCI Species List from Literature.json", "r") as f:
    sp_list = json.loads(f.read())

In [30]:
#This could be improved upon and worked more closely into bispy logic

sb_data_release_cache = []
data_fields = "id,title,body"

for name in sp_list:
    try:
        data = sb.data_release_search(name, data_fields)
        if data:
            sp_data = {name: data}
            sb_data_release_cache.append(sp_data)
    except:
        #sb_data_release_cache.append({name: 'failed to query sciencebase'})
        pass


In [29]:
sb_data_release_cache

[]

In [26]:
# Dump just the records we found to a cache file
with open("cache/sb_data_release.json", "w") as f:
    f.write(json.dumps([s for s in sb_data_release_cache], indent=4))

In [27]:
# Open the file back up and verify
with open("cache/sb_data_release.json", "r") as f:
    sb_data_release_cache = json.loads(f.read())

print(len(sb_data_release_cache))
display(sb_data_release_cache[random.randint(0,len(sb_data_release_cache)-1)])

0


ValueError: empty range for randrange() (0,0, 0)

Need to finialize this.  We need to add a variation of this function to bispy, compile this information to the cache and write out the info.

#This function was added into the bispy package, but pull request is pending so I am leaving this here until it is accepted.
def sciencebase_search(term):
    term = term.replace(" ", "%20")
    base_url = "https://www.sciencebase.gov/catalog/items"
    #parameters searching term for those items marked as Official Data Release or Data Release in Progress
    parameters = "?q=" + term + "&filter=systemType%3DData+Release&filter=browseCategory%21%3DData+Release+-+In+Progress&format=json&max=50" 
    search_url = base_url + parameters
    print (search_url)
    r = requests.get(search_url).json()
    return r